## Train Test Split

Choose 20% of genes to be the unseen test set.
For each test gene, remove all its transcripts from the training set.

The task is simplified by our upstream data processing.
Instead of one csv row per (gene_id,cell_line), we have one row per (gene_id).

This notebook reads lines of text, shuffles them, partitions them, writes them. The code ignores the csv nature of each line.
However, the code takes the header line from input and repeats it in each output.


In [ ]:
from random import Random

TEST_PORTION = 0.2
ATLAS_DIR='./data/LncAtlas/'
ATLAS_CODING='CNRCI_coding_genes.csv'
ATLAS_NONCODING='CNRCI_noncoding_genes.csv'
GENCODE_DIR='./data/GenCode/'
GENCODE_CODING='Homo_sapiens.GRCh38.cds.csv'
GENCODE_NONCODING='Homo_sapiens.GRCh38.ncrna.csv'

CODING_TEST = './data/CDHIT/CNRCI_coding_test_genes.csv'
CODING_TRAIN = './data/CDHIT/CNRCI_coding_train_genes.csv'
NONCODING_TEST = 'CNRCI_noncoding_test_genes.csv'
NONCODING_TRAIN = 'CNRCI_noncoding_train_genes.csv'

In [ ]:
def load_all_rows(filepath):
    with open (filepath,'r') as handle:
        all_rows = handle.readlines()
    header = all_rows[0]
    data = all_rows[1:]
    return header,data

In [ ]:
header_coding,all_coding_rows = load_all_rows(ATLAS_DIR+ATLAS_CODING)
header_noncoding,all_noncoding_rows = load_all_rows(ATLAS_DIR+ATLAS_NONCODING)

In [ ]:
all_coding_rows

In [ ]:
import pandas as pd
atlas_coding = pd.read_csv("./data/CDHIT/CNRCI_coding_genes.csv")
gene_coding = pd.read_csv("./data/CDHIT/all_hg38.cds.csv")

# atlas_noncoding = pd.read_csv("./data/LncAtlas/CNRCI_noncoding_genes.csv")
# gene_noncoding = pd.read_csv("./data/GenCode/Homo_sapiens.GRCh38.ncrna.csv")


In [ ]:
atlas_coding.head(2)

gene_id     A549  H1.hESC  HeLa.S3    HepG2    HT1080    HUVEC  \
0  ENSG00000000003  1.08068  1.85734  1.86839  2.29436  0.866395  1.28494   
1  ENSG00000000005      NaN  5.88264      NaN      NaN       NaN      NaN   

     MCF.7  NCI.H460      NHEK  SK.MEL.5  SK.N.DZ  SK.N.SH  GM12878  K562  \
0  1.72696  -0.26651  0.460806   1.83753 -1.11901  1.83443      NaN   NaN   
1      NaN       NaN       NaN       NaN      NaN      NaN      NaN   NaN   

   IMR.90  
0     NaN  
1     NaN

## <font color=blue> Remove genes without sequences in GeneCode</font>

In [ ]:
len(atlas_coding),len(gene_coding),len(set(gene_coding["gene_id"]) & set(atlas_coding["gene_id"]))

(17770, 24503, 15835)

In [ ]:
# len(atlas_noncoding),len(gene_noncoding),len(set(gene_noncoding["gene_id"]) & set(atlas_non_coding["gene_id"]))

(6768, 55625, 5777)

In [ ]:
atlas_coding_ = atlas_coding[atlas_coding["gene_id"].isin(gene_coding["gene_id"])]
# atlas_noncoding_ = atlas_noncoding[atlas_noncoding["gene_id"].isin(gene_noncoding["gene_id"])]
len(atlas_coding_)

15835

In [ ]:
atlas_coding_.to_csv("./data/CDHIT/atlas_coding_genes.csv",index=False)
# atlas_noncoding_.to_csv("./data/CDHIT/atlas_noncoding_genes.csv",index = False)


In [ ]:
df = pd.read_csv("./data/CDHIT/atlas_coding_genes.csv")
df.head(2)

gene_id     A549  H1.hESC  HeLa.S3    HepG2    HT1080    HUVEC  \
0  ENSG00000000003  1.08068  1.85734  1.86839  2.29436  0.866395  1.28494   
1  ENSG00000000005      NaN  5.88264      NaN      NaN       NaN      NaN   

     MCF.7  NCI.H460      NHEK  SK.MEL.5  SK.N.DZ  SK.N.SH  GM12878  K562  \
0  1.72696  -0.26651  0.460806   1.83753 -1.11901  1.83443      NaN   NaN   
1      NaN       NaN       NaN       NaN      NaN      NaN      NaN   NaN   

   IMR.90  
0     NaN  
1     NaN

In [ ]:
header_coding,all_coding_rows = load_all_rows("./data/CDHIT/atlas_coding_genes.csv")
# header_noncoding,all_noncoding_rows = load_all_rows("./data/lncAtlas/atlas_noncoding_genes.csv")

In [ ]:
len(all_coding_rows)

15835

In [ ]:
def inplace_shuffle(rows):
    generator = Random()
    generator.seed(42)
    generator.shuffle(rows)  # in-place
inplace_shuffle(all_coding_rows)
inplace_shuffle(all_coding_rows)

In [ ]:
def train_test_split(rows):
    length = len(rows)
    divider = int(length*TEST_PORTION)
    train_set = rows[divider:]
    test_set = rows[:divider]
    return (train_set,test_set)
coding_train_set,coding_test_set = train_test_split(all_coding_rows)
# noncoding_train_set,noncoding_test_set = train_test_split(all_noncoding_rows)

In [ ]:
def save_csv(header,rows,filepath):
    with open(filepath,'w') as handle:
        handle.write(header)
        for line in rows:
            handle.write(line)

In [ ]:
filename = CODING_TEST
save_csv( header_coding, coding_test_set, filename)

filename = CODING_TRAIN
save_csv( header_coding, coding_train_set, filename)

# filename = ATLAS_DIR + NONCODING_TEST
# save_csv( header_noncoding, noncoding_test_set, filename)

# filename = ATLAS_DIR + NONCODING_TRAIN
# save_csv( header_noncoding, noncoding_train_set, filename)

In [ ]:
print('done')

done


## <font color=blue> comparison</font>

In [ ]:
print('   Coding total, train, test:',
      len(all_coding_rows),len(coding_train_set),len(coding_test_set))
# print('Noncoding total, train, test:',
#       len(all_noncoding_rows),len(noncoding_train_set),len(noncoding_test_set))

   Coding total, train, test: 15835 12668 3167


## <font color=blue> comparison</font>

In [ ]:
print('   Coding total, train, test:',
      len(all_coding_rows),len(coding_train_set),len(coding_test_set))
print('Noncoding total, train, test:',
      len(all_noncoding_rows),len(noncoding_train_set),len(noncoding_test_set))

   Coding total, train, test: 17770 14216 3554
Noncoding total, train, test: 6768 5415 1353
